In [1]:
import pandas as pd
import requests, zipfile, io, os, re
import tkinter as tk
import geopandas as gpd
import datetime as dt
from shapely.geometry import Point, LineString
from tkinter import filedialog
%matplotlib inline

In [2]:
months=list(range(1,13))
years=list(range(2019,(dt.datetime.now().year)+1))

In [3]:
def button_infolder_clicked():
    root.infolder=filedialog.askdirectory(initialdir="/",title="Select Input folder")
def button_outfolder_clicked():
    root.outfolder=filedialog.askdirectory(initialdir="/",title="Select Output folder")
def close_window():
    root.start_month=tkvar_start_month.get()
    root.start_year=tkvar_start_year.get()
    root.end_month=tkvar_end_month.get()
    root.end_year=tkvar_end_year.get()
    root.destroy()

root=tk.Tk()
root.title('Emerging Mobility')
# root.geometry('350x200')

tkvar_start_month=tk.StringVar(root)
tkvar_start_year=tk.StringVar(root)
tkvar_end_month=tk.StringVar(root)
tkvar_end_year=tk.StringVar(root)

label_in=tk.Label(root,text='Select Folder').grid(column=0,row=0)
button_in=tk.Button(root,text="Browse",command=button_infolder_clicked).grid(column=1,row=0)

label_out=tk.Label(root,text="Select an output folder").grid(column=0,row=1)
button_out=tk.Button(root,text="Browse",command=button_outfolder_clicked).grid(column=1,row=1)

start_month_label=tk.Label(root,text="Select start month").grid(column=0,row=2)
start_month_choice=tk.OptionMenu(root,tkvar_start_month,*months).grid(column=1,row=2)

start_year_label=tk.Label(root,text="Select start year").grid(column=0,row=3)
start_year_choice=tk.OptionMenu(root,tkvar_start_year,*years).grid(column=1,row=3)

end_month_label=tk.Label(root,text="Select end month").grid(column=0,row=4)
end_month_choice=tk.OptionMenu(root,tkvar_end_month,*months).grid(column=1,row=4)

end_year_label=tk.Label(root,text="Select end year").grid(column=0,row=5)
year_choice=tk.OptionMenu(root,tkvar_end_year,*years).grid(column=1,row=5)

button_run=tk.Button(root,text='Run',width=25,command=close_window).grid(column=0,row=6)
root.mainloop()

In [4]:
start_month=dt.date(int(root.start_year),int(root.start_month),1)
start_month=start_month.strftime("%Y-%m-%d")
end_month=dt.date(int(root.end_year),int(root.end_month),1)
end_month=end_month.strftime("%Y-%m-%d")

In [5]:
save_file=root.outfolder
save_file=save_file+r"/{}.shp"

In [6]:
idaho='16'
ada_county='001'
canyon_county='027'
tracts_url=r"https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_16_tract.zip"

In [7]:
def readOnlineShapefile(url):
    zip_file_url=url
    r=requests.get(zip_file_url)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("x")
    return gpd.read_file("x")
tracts=readOnlineShapefile(tracts_url)

In [8]:
def getVRTServiceArea(url):
    gdf=readOnlineShapefile(url)
    gdf=gdf.loc[(gdf['COUNTYFP']==ada_county)|(gdf['COUNTYFP']==canyon_county)]
    return gdf
tracts=getVRTServiceArea(tracts_url)

In [9]:
FMLM_Folder=r"N:\Planning - New File Structure\GIS\Data\EmergingMobility\FMLM"
NiteRide_Folder=r"N:\Planning - New File Structure\GIS\Data\EmergingMobility\NiteRide"

In [10]:
def mobilityReader(folder,start_month,end_month,save_file=None,output_type='line'):
    save_name=os.path.basename(folder)+"_"+start_month[:7]+"_"+end_month[:7]+"_"+output_type
    df=pd.DataFrame()
    for i in os.listdir(folder):
        df=df.append(pd.read_excel(folder+"\\"+i,dtype={"Origin (Census Tract)":str,"Destination (Census Tract)":str},sheet_name='Invoice'))
    df=df.loc[(df['Month/Year']>=start_month)&(df['Month/Year']<=end_month)]
    
    tracts['center']=tracts['geometry'].centroid
    tracts['lon']=tracts.center.apply(lambda p: p.x)
    tracts['lat']=tracts.center.apply(lambda p: p.y)
    df_origins=df.merge(tracts,left_on="Origin (Census Tract)",right_on='GEOID')
    df_origins.drop(['center','geometry'],axis=1,inplace=True)
    df_dests=df.merge(tracts,left_on="Destination (Census Tract)",right_on="GEOID")
    df_dests.drop(['center','geometry'],axis=1,inplace=True)
    df=df_origins.join(df_dests,lsuffix='_orig',rsuffix='_dest')
    
    df['orig_coord']=[Point(xy) for xy in zip(df.lon_orig,df.lat_orig)]
    df['dest_coord']=[Point(xy)for xy in zip(df.lon_dest,df.lat_dest)]
    
    df=df[['Passenger ID_orig','Day of Travel_orig','Dispatch Method_orig','Coupon Name_orig',"Origin (Census Tract)_orig","Destination (Census Tract)_dest",'Trip Time Period_orig','Trip Length_orig','Trip Duration_orig','Trip Cost_orig','Trip Refund_orig','Trip Subsidy_orig','lon_orig','lat_orig','lon_dest','lat_dest','orig_coord','dest_coord']]
    df['line']=[LineString(xy) for xy in zip(df.orig_coord,df.dest_coord)]
    df['trips']=1
    
    gdf=gpd.GeoDataFrame(df,geometry='line',crs={'init':'epsg:4326'})
    gdf.drop(['orig_coord','dest_coord','lon_orig','lat_orig','lon_dest','lat_dest'],axis=1,inplace=True)
    gdf.rename(columns={"Origin (Census Tract)_orig":'Origin',"Destination (Census Tract)_dest":'Destination'},inplace=True)
    
    gdf2=gdf[['Origin','Destination','Trip Time Period_orig','line','trips']]
    gdf2=gdf2.dissolve(['Origin','Destination','Trip Time Period_orig'],aggfunc='sum')
    gdf2.reset_index(inplace=True)
    
    gdf_point=gdf2.loc[gdf2['Origin']==gdf2['Destination']]
    gdf_point=gdf_point.merge(tracts,left_on='Origin',right_on='GEOID')
    gdf_point=gpd.GeoDataFrame(gdf_point,geometry='center',crs={'init':'epsg:4326'})
    
    gdf_line=gdf2.loc[gdf2['Origin']!=gdf2['Destination']]
    gdf_point=gdf_point[['Origin','Destination','Trip Time Period_orig','trips','center']]
    gdf_line=gdf_line[['Origin','Destination','Trip Time Period_orig','trips','line']]
    if output_type=='line':
        if save_file:
            gdf_line.to_file(save_file.format(save_name))
        return gdf_line
    if output_type=='point':
        if save_file:
            gdf_point.to_file(save_file.format(save_name),SHPT='POINT')
        return gdf_point
mobilityReader(root.infolder,start_month=start_month,end_month=end_month,output_type='line',save_file=save_file)
mobilityReader(root.infolder,start_month=start_month,end_month=end_month,output_type='point',save_file=save_file)

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


,Origin,Destination,Trip Time Period_orig,trips,center
0,16001000304,16001000304,AM PEAK,11,POINT (-116.2649956409961 43.66851120605958)
1,16001000304,16001000304,MIDDAY,36,POINT (-116.2649956409961 43.66851120605958)
2,16001000304,16001000304,PM PEAK,33,POINT (-116.2649956409961 43.66851120605958)
3,16001000500,16001000500,AM PEAK,2,POINT (-116.2232440627798 43.64041742720104)
4,16001000500,16001000500,PM PEAK,1,POINT (-116.2232440627798 43.64041742720104)
5,16001002302,16001002302,AM PEAK,14,POINT (-116.2847753565571 43.61722615312507)
6,16001002302,16001002302,MIDDAY,91,POINT (-116.2847753565571 43.61722615312507)
7,16001002302,16001002302,PM PEAK,106,POINT (-116.2847753565571 43.61722615312507)
8,16001002312,16001002312,AM PEAK,6,POINT (-116.3042485191839 43.62668193670114)
9,16001002312,16001002312,MIDDAY,18,POINT (-116.3042485191839 43.62668193670114)
